In [ ]:
! pip install featuretools

In [17]:
import pandas as pd
import numpy as np
import ast
import re

In [18]:
train_1 = pd.read_csv('train_1.csv', low_memory=False)

In [ ]:
train_1.dtypes

In [ ]:
list_str_obj_cols = train_1.columns[train_1.dtypes == "object"].tolist()
for str_obj_col in list_str_obj_cols:
    train_1[str_obj_col] = train_1[str_obj_col].astype("category")

list_str_obj_cols = ['loan_id', 'id']
for str_obj_col in list_str_obj_cols:
    train_1[str_obj_col] = train_1[str_obj_col].astype("object")

list_str_obj_cols = ['label']
for str_obj_col in list_str_obj_cols:
    train_1[str_obj_col] = train_1[str_obj_col].astype("bool")

train_1.dtypes

In [ ]:
train_1['label'].value_counts()

In [22]:
train_1.to_pickle('train_1_proc.bin')

In [23]:
train_2_1 = pd.read_csv('train_2_1.csv', low_memory=False)

In [ ]:
train_2_1[train_2_1.columns[train_2_1.dtypes == "object"]]

In [ ]:
list_str_obj_cols = ['add_431']
for str_obj_col in list_str_obj_cols:
    if str_obj_col in train_2_1.columns:
        train_2_1[str_obj_col] = pd.to_datetime(train_2_1[str_obj_col], errors='coerce')
        
        # Add year, month, day, weekday, and weekend columns
        train_2_1[f'{str_obj_col}_year'] = train_2_1[str_obj_col].dt.year
        train_2_1[f'{str_obj_col}_month'] = train_2_1[str_obj_col].dt.month
        train_2_1[f'{str_obj_col}_day'] = train_2_1[str_obj_col].dt.day
        train_2_1[f'{str_obj_col}_weekday'] = train_2_1[str_obj_col].dt.weekday
        train_2_1[f'{str_obj_col}_is_weekend'] = train_2_1[str_obj_col].dt.weekday >= 5

        # Drop the original date column
        train_2_1.drop(columns=[str_obj_col], inplace=True)

list_str_obj_cols = ['add_671', 'add_672', 'add_673', 'add_675', 'add_676', 'add_677']

def safe_eval_array(x):
    try:
        x = re.sub(r'\b0+(\d)', r'\1', x)
        x = ast.literal_eval(x)
        return ast.literal_eval(x)
    except:
        return []

def unwrap_arrays(data, columns):
    for col in columns:
        # Step 1: Convert string to lists using safe_eval_array
        mySeries = data[col].values
        mySeries = mySeries.tolist()
        mySeries = [safe_eval_array(x) for x in mySeries]
        mySeries = pd.Series(mySeries)
        # print(col)
        # print(mySeries)

        # Step 3: Find the maximum length of the arrays in the column
        max_len = mySeries.apply(len).max()

        # Step 4: Create separate columns for each element in the arrays
        exploded_cols = pd.DataFrame(
            mySeries.apply(lambda x: x + [np.nan] * (max_len - len(x))).tolist(),
            index=data.index,
            columns=[f'{col}_elem_{i+1}' for i in range(max_len)]
        )

        # Step 5: Calculate the mean and length of the arrays
        data[f'{col}_mean'] = mySeries.apply(lambda x: np.mean(x) if len(x) > 0 else np.nan)
        data[f'{col}_len'] = mySeries.apply(len)

        # Step 6: Concatenate the exploded columns to the original dataframe
        data = pd.concat([data, exploded_cols], axis=1)

        # Step 7: Drop the original column
        data.drop(columns=[col], inplace=True)

    return data

train_2_1 = unwrap_arrays(train_2_1, list_str_obj_cols)

train_2_1.dtypes

In [26]:
train_2_1.to_pickle('train_2_1_proc.bin')

In [27]:
train_2_1 = pd.read_csv('train_2_2.csv', low_memory=False)

In [ ]:
train_2_1[train_2_1.columns[train_2_1.dtypes == "object"]]

In [ ]:
list_str_obj_cols = ['add_431']
for str_obj_col in list_str_obj_cols:
    if str_obj_col in train_2_1.columns:
        train_2_1[str_obj_col] = pd.to_datetime(train_2_1[str_obj_col], errors='coerce')
        
        # Add year, month, day, weekday, and weekend columns
        train_2_1[f'{str_obj_col}_year'] = train_2_1[str_obj_col].dt.year
        train_2_1[f'{str_obj_col}_month'] = train_2_1[str_obj_col].dt.month
        train_2_1[f'{str_obj_col}_day'] = train_2_1[str_obj_col].dt.day
        train_2_1[f'{str_obj_col}_weekday'] = train_2_1[str_obj_col].dt.weekday
        train_2_1[f'{str_obj_col}_is_weekend'] = train_2_1[str_obj_col].dt.weekday >= 5

        # Drop the original date column
        train_2_1.drop(columns=[str_obj_col], inplace=True)

list_str_obj_cols = ['add_671', 'add_672', 'add_673', 'add_675', 'add_676', 'add_677']

train_2_1 = unwrap_arrays(train_2_1, list_str_obj_cols)

train_2_1.dtypes

In [30]:
train_2_1.to_pickle('train_2_2_proc.bin')